In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from mvf_bto.data_loading import load_data
from mvf_bto.constants import * 
from mvf_bto.models.convolutional_1d import Convolutional1D
from mvf_bto.preprocessing.convolutional import create_discharge_inputs
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError
from scipy.interpolate import interp1d

import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go

## Loading Data

In [ ]:
data_path = "/Users/anoushkabhutani/PycharmProjects/10701-mvf-bto/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"
# data_path = "/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto-backup/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"

In [ ]:
data = load_data(file_path=data_path, num_cells=15)

## Preprocessing to create model inputs and targets

In [ ]:
train_split = 0.7
test_split = 0.2
forecast_horizon=4
history_window=10

In [ ]:
# by default uses validation_split = 1 - (train_split + test_split)
datasets = create_discharge_inputs(data, train_split, test_split, 
                                   forecast_horizon=forecast_horizon,
                                   history_window=history_window)

## Train Model

In [ ]:
window_length = datasets["X_train"].shape[1]
n_features = datasets['n_features']
input_shape = (window_length, n_features)
output_dimension = datasets["y_train"].shape[-1]

In [ ]:
from tensorflow import keras

In [ ]:
model = Convolutional1D(input_shape=input_shape, n_outputs=output_dimension)

es = EarlyStopping(
    monitor="val_mean_squared_error",
    min_delta=0,
    patience=40,
    verbose=1,
    mode="min",
    restore_best_weights=True,
)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.96)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss="mse", 
              metrics=[MeanSquaredError()])

history = model.fit(
    datasets["X_train"],
    datasets["y_train"],
    validation_data=(datasets["X_val"], datasets["y_val"]),
    epochs=300,
    callbacks=[es],
    batch_size=128,
    shuffle=False,
    verbose=1,

)

## Parity Plots

In [ ]:
model.summary()

In [ ]:
pred_df = pd.DataFrame(model.predict(datasets["X_test"], verbose=0))
test_df = pd.DataFrame(datasets["y_test"])
column_names = []
for i in range(len(test_df.columns)//2):
    column_names.append(f"Voltage (FH = {i+1})")
    column_names.append(f"Temperature (FH = {i+1})")
pred_df.columns =  column_names
test_df.columns =  column_names

In [ ]:
skip = 30
parity_plot_list = []
for column in test_df.columns:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="lines"))
    fig.add_trace(go.Scatter(x = pred_df[column][::skip], showlegend=False,  y=test_df[column][::skip], mode="markers"))
    fig.update_yaxes(title=f"Normalized Target {column}")
    fig.update_xaxes(title=f"Normalized Prediction {column}")
    fig.update_layout(template="simple_white")
    parity_plot_list.append(fig)

In [ ]:
parity_plot_list[0]

In [ ]:
parity_plot_list[7]

In [ ]:
parity_plot_list[-1]

## True vs Predicted Traces

In [ ]:
cycles = [int(datasets["X_test"][i][-1][-1]*MAX_CYCLE) for i in range(len(datasets["X_test"]))]
pred_df = pd.DataFrame(model.predict(datasets["X_test"], verbose=1))
test_df = pd.DataFrame(datasets["y_test"])
column_names = []
for i in range(len(test_df.columns)//2):
    column_names.append(f"Voltage (FH = {i+1})")
    column_names.append(f"Temperature (FH = {i+1})")
pred_df.columns =  column_names
test_df.columns =  column_names
test_df['Cycle']= cycles
pred_df['Cycle']=cycles
cycles_plot = pred_df['Cycle'].unique()[::30]
pred_df['Q'] = [datasets["X_test"][i][0][1]*1.6 for i in range(0,len(datasets["X_test"]))]
test_df['Q'] = [datasets["X_test"][i][0][1]*1.6 for i in range(0,len(datasets["X_test"]))]
test_cell_ids = datasets['original_test'].Cell.unique()

In [ ]:
pallete = plotly.colors.qualitative.Dark24 + plotly.colors.qualitative.T10
pallete = pallete*70000
pred_df.columns

In [ ]:
split_indices = [0,]
split_indices.extend([i[0] for i in np.where(np.diff(pred_df.Cycle)<0)])
split_indices.append(len(pred_df))
cell_wise_pred_dfs = [pred_df.iloc[split_indices[n]:split_indices[n+1]] for n in range(len(split_indices)-1)]
cell_wise_test_dfs = [test_df.iloc[split_indices[n]:split_indices[n+1]] for n in range(len(split_indices)-1)]

In [ ]:
traces_figures= []
X_test = datasets["X_test"][:47*2]
y_test = datasets["y_test"][:47*2]
y_hat = model.predict(X_test)
for column in pred_df.columns:
    fig = go.Figure()
    for cycle in cycles_plot:
        for idx in range(len(cell_wise_pred_dfs)):
            pred_df = cell_wise_pred_dfs[idx]
            test_df = cell_wise_test_dfs[idx]
            fig.add_trace(go.Scatter(x = [i[3][1] for i in X_test], y =[i[-2] for i in y_test], 
                                     name = f"Cycle {cycle} Cell {test_cell_ids[idx]}",
                                     mode="lines+markers", line_color = pallete[cycle], ))
            fig.add_trace(go.Scatter(x = [i[3][1] for i in X_test[10:]], y=[i[-2] for i in y_hat], name = f"Prediction",
                                         mode="markers", line_color = pallete[cycle], ))
        fig.update_xaxes(title="SOC")
        fig.update_yaxes(title=column)
    traces_figures.append(fig)

In [ ]:
n_cycles = int(datasets['X_test'].shape[0]/datasets['arrays_per_cycle'])

fig_list = []

for nf_steps in range(0, forecast_horizon*2, 2):
    fig = go.Figure()
    for i in range(0, n_cycles, 40):
        X_cycle = datasets['X_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        y_cycle = datasets['y_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        y_hat = model.predict(X_cycle, verbose=0)
        fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                 y =[i[nf_steps]  for i in y_cycle], 
                                 mode="lines", line_color = pallete[i], name="Data"))
        fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                 y =[i[nf_steps] for i in y_hat], 
                                 mode="markers", line_color = pallete[i], name= "Prediction"))
    fig.update_xaxes(title="Normalized Capacity [Ah]")
    fig.update_yaxes(title="Normalized Voltage [V]")
    fig.update_layout(title=f"Voltage Prediction at Forecast Horizon {int(nf_steps//2)+1}")
    fig_list.append(fig)

In [ ]:
for nf_steps in range(1, forecast_horizon*2, 2):
    fig = go.Figure()
    for i in range(0, n_cycles, 40):
        X_cycle = datasets['X_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        y_cycle = datasets['y_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        y_hat = model.predict(X_cycle, verbose=0)
        fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                 y =[i[nf_steps]  for i in y_cycle], 
                                 mode="lines", line_color = pallete[i], name="Data"))
        fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                 y =[i[nf_steps] for i in y_hat], 
                                 mode="markers", line_color = pallete[i], name= "Prediction"))
    fig.update_xaxes(title="Normalized Capacity [Ah]")
    fig.update_yaxes(title="Normalized Temperature [°C]")
    fig.update_layout(title=f"Temperature Prediction at Forecast Horizon {int(nf_steps//2)+1}")
    fig_list.append(fig)

In [ ]:
fig_list[0]

In [ ]:
fig_list[2]

In [ ]:
fig_list[-1]

In [ ]:
fig_list[-3]

In [ ]:
model.summary()